In [1]:
import os
import pandas as pd
import numpy as np
import wrds
from random import *
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats
import matplotlib.pyplot as plt
import pandas_datareader
import datetime
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
data_folder = '/Users/chengxinxiangye/Desktop/Quant Asset/' 
id_wrds = 'yahan123456'  
possible_exchcd = (1, 2, 3)
possible_shrcd = (10,11)

In [4]:
# Establish a connection to WRDS
conn = wrds.Connection(wrds_username='wrds_id')

# Load CRSP returns and change variables format
crsp_raw = conn.raw_sql("""
    select a.permno, a.date, b.shrcd, b.exchcd, 
    a.ret, a.prc, a.shrout
    from crspq.msf as a
    left join crsp.msenames as b
    on a.permno=b.permno
    and b.namedt<=a.date
    and a.date<=b.nameendt
    where a.date between '01/01/1900' and '12/31/2023'
""")

# Sort data
crsp_raw = crsp_raw.sort_values(['permno', 'date']).reset_index(drop=True).copy()

# Set date column to datetime
crsp_raw['date'] = pd.to_datetime(crsp_raw['date'], format='%Y-%m-%d', errors='ignore') + MonthEnd(0)

# Store data to data folder
crsp_raw.to_pickle(data_folder + 'crsp_raw.pkl')
dlret_raw = conn.raw_sql("""
    select permno, dlret, dlstdt, dlstcd
    from crspq.msedelist
""")

# Set date column to datetime
dlret_raw['date'] = pd.to_datetime(dlret_raw['dlstdt'])+ MonthEnd(0)

# Store data to data folder
dlret_raw.to_pickle(data_folder + 'dlret_raw.pkl')

KeyboardInterrupt: Interrupted by user

In [4]:
dlret_raw = conn.raw_sql("""
    select permno, dlret, dlstdt, dlstcd
    from crspq.msedelist
""")

# Set date column to datetime
dlret_raw['date'] = pd.to_datetime(dlret_raw['dlstdt'])+ MonthEnd(0)

# Store data to data folder
dlret_raw.to_pickle(data_folder + 'dlret_raw.pkl')

In [8]:
#Load Data from folder
dcrsp_raw = pd.read_pickle(data_folder + 'crsp_raw.pkl')

# Makesure data is in numeric type
dcrsp_raw['permno'] = dcrsp_raw['permno'].astype(int)

# Load CRSP Deslisting returns
dlret_raw = pd.read_pickle(data_folder + 'dlret_raw.pkl')

dlret_raw['date'] = pd.to_datetime(dlret_raw['dlstdt'])+ MonthEnd(0)
# Makesure data is in numeric type
dlret_raw['permno'] = dlret_raw['permno'].astype(int)

# Merge for input
dcrsp = dcrsp_raw.merge(dlret_raw[['dlret','date','permno']], how='left', on=['permno','date'])
dcrsp1 = dcrsp[['permno','date','exchcd','ret','dlret','prc','shrout']].copy()

In [10]:
CRSP_Stocks = dcrsp1.copy()
CRSP_Stocks

,permno,date,exchcd,ret,dlret,prc,shrout
0,10000,1986-01-31,3,NaN,NaN,-4.37500,3680.0
1,10000,1986-02-28,3,-0.257143,NaN,-3.25000,3680.0
2,10000,1986-03-31,3,0.365385,NaN,-4.43750,3680.0
3,10000,1986-04-30,3,-0.098592,NaN,-4.00000,3793.0
4,10000,1986-05-30,3,-0.222656,NaN,-3.10938,3793.0
...,...,...,...,...,...,...,...
3772178,93436,2023-11-30,3,0.195379,NaN,240.08000,3178921.0
3772179,93436,2023-12-29,3,0.034988,NaN,248.48000,3185000.0
3772180,93436,2024-01-31,3,-0.246257,NaN,187.28999,3184790.0
3772181,93436,2024-02-29,3,0.077901,NaN,201.88000,3184790.0


In [7]:
def PS1_Q1(dcrsp):
    # Filter data by SHRCD, EXCHCD, and date
    dcrsp=dcrsp[dcrsp['exchcd'].isin(possible_exchcd)]
    dcrsp=dcrsp[dcrsp['shrcd'].isin(possible_shrcd)]
    
    #dcrsp['permno'] = dcrsp['permno'].astype(int)
    # Adjust delisting return
    # 1. If both delisting and holding period return are not NA, calculate return using both
    dcrsp['ret'] = np.where(dcrsp['ret'].notna() & dcrsp['dlret'].notna(), 
                        (1+dcrsp['ret'])*(1+dcrsp['dlret'])-1, dcrsp['ret'])
    # 2. If holding period return is NA and delisting return is not NA, set return to be delisting return
    dcrsp['ret'] = np.where(dcrsp['ret'].isna()  & dcrsp['dlret'].notna(), dcrsp['dlret'], dcrsp['ret'])
    # 3. Dropping missing returns
    dcrsp = dcrsp[dcrsp['ret'].notna()].copy() 
    
    # Calculate Market Equity = Price($) x shares outstanding(thousands) / thousands (in billion)
    dcrsp['me'] = dcrsp['prc'].abs() * dcrsp['shrout'] * 1e-3 
    
    dcrsp = dcrsp[['date','permno','ret','me']].sort_values(by=['date','permno']).reset_index(drop=True).copy()   
    dcrsp = dcrsp[dcrsp['me'].notna()].copy() 
    
    # Calculate lagged market equity
    dcrsp['Lme'] = dcrsp[['permno','me']].groupby('permno').shift(1)
    
    dcrsp = dcrsp[dcrsp['Lme'] > 0].reset_index(drop=True).copy()
    
    # Sort date by date
    dcrsp = dcrsp.sort_values(['date']).reset_index(drop=True).copy()
    
    # Calculate value weight using lagged market equity 
    dcrsp = dcrsp.merge(dcrsp[['date','Lme']].groupby(['date']).sum()\
                        .reset_index().rename(columns={"Lme":"LmeTotal"}),on=['date'],
                        how='left')

    # Calculate monthly value weighted return for each stock
    dcrsp['vwret1m'] = dcrsp['ret'] * (dcrsp['Lme'] / dcrsp['LmeTotal']) 

    # Calculate monthly vw, lagged return by sum monthly per stocks return
    # Calculate equal weighted by simply take average of return in one month for all stocks
    dcrsp = dcrsp[['date', 'vwret1m', 'Lme', 'ret']].groupby(['date'])\
                            .agg({'vwret1m': 'sum','Lme': 'sum','ret':'mean'}).reset_index()\
                            .rename(columns={'vwret1m': 'Stock_Vw_Ret', 'Lme': 'Stock_lag_MV', 'ret' : "Stock_Ew_Ret" })
    
    dcrsp = dcrsp[(dcrsp['date'].dt.year>1925) & (dcrsp['date'].dt.year<2024)].copy()    
    # Creat Year Variable for output
    dcrsp['Year'] = dcrsp['date'].dt.year
    
    # Creat Month Variable for output
    dcrsp['Month'] = dcrsp['date'].dt.month
    
    # Output
    dcrsp = dcrsp[['Year','Month','Stock_lag_MV',"Stock_Ew_Ret",'Stock_Vw_Ret']]
    
    return dcrsp

In [8]:
Monthly_CRSP_Stocks = PS1_Q1(CRSP_Stocks)
# Store data to data folder
Monthly_CRSP_Stocks.to_pickle(data_folder + 'Monthly_CRSP_Stocks.pkl')
print(Monthly_CRSP_Stocks.dtypes)

Year              int32
Month             int32
Stock_lag_MV    float64
Stock_Ew_Ret    float64
Stock_Vw_Ret    float64
dtype: object


In [9]:
print(Monthly_CRSP_Stocks.head(5))

   Year  Month  Stock_lag_MV  Stock_Ew_Ret  Stock_Vw_Ret
0  1926      2  26789.658062     -0.054045     -0.033337
1  1926      3  25996.476438     -0.096528     -0.064638
2  1926      4  24078.318125      0.031147      0.036776
3  1926      5  25235.879875      0.000905      0.012319
4  1926      6  25244.145625      0.049799      0.054652


## Q2

In [10]:
# Download FF data
FF3 = pandas_datareader.famafrench.FamaFrenchReader('F-F_Research_Data_Factors', start='1926', end='2024')
FF3 = FF3.read()[0] / 100  # Monthly data
FF3.columns = ['MktRF', 'SMB', 'HML', 'RF']
FF3 = FF3.reset_index().rename(columns={"Date": "date"}).copy()

# Set date column to datetime format
FF3['date'] = pd.DataFrame(FF3[['date']].values.astype('datetime64[ns]')) + MonthEnd(0)
# Prepare for input
# Creat Year Variable
FF3['Year'] = FF3['date'].dt.year   
# Creat Month Variable
FF3['Month'] = FF3['date'].dt.month

#Only keep columns needed
FF3 = FF3[['Year','Month','MktRF','SMB','HML','RF']].copy()

In [12]:
# define a function to calculate required statistics
def calulation(data):
    AM = round(data.mean()*1200,2) # in percentage
    AS = round(data.std()*np.sqrt(12)*100,2) # in percentage
    SR = round(AM/AS,2)
    SK = round(data.skew(),2)
    KT = round(data.kurtosis(),2)
    return [AM,AS,SR,SK,KT]

In [13]:
def PS1_Q2(Q1,FF):
    # Specify requried column names and row names
    rowname = ['Annualized Mean(%)', 'Annualized Standard Deviation(%)',\
               'Annualized Sharpe Ratio', 'Skewness','Excess Kurtosis']
    colname = ['Estimated FF Market Excess Return', 'Actual FF Market Excess Return']
    # Creat a dataframe with row,colnames
    df = pd.DataFrame(np.nan, index=rowname, columns=colname)
    
    # Merge two times series and keep all columns in both datasets
    Q2 = pd.merge(Q1,FF,on=['Month','Year'],how='outer').copy()
    
    # Calculate estimated excess returns
    Q2['excess'] = Q2['Stock_Vw_Ret'] - Q2['RF']
    
    # Make sure only include data from July 1926 to December 2023
    start = np.where((Q2['Year'] == 1926) & (Q2['Month'] == 7))[0][0]
    end = np.where((Q2['Year'] == 2023) & (Q2['Month'] == 12))[0][0]
    Q2 = Q2[start:(end+1)].copy()
    df['Estimated FF Market Excess Return'] = calulation(Q2['excess'])
    df['Actual FF Market Excess Return'] = calulation(Q2['MktRF'])
    return df

In [14]:
PS1_Q2(Monthly_CRSP_Stocks,FF3)

,Estimated FF Market Excess Return,Actual FF Market Excess Return
Annualized Mean(%),8.14,8.14
Annualized Standard Deviation(%),18.50,18.51
Annualized Sharpe Ratio,0.44,0.44
Skewness,0.16,0.16
Excess Kurtosis,7.44,7.41


In [15]:
def PS1_Q3(Q1,FF):
    Q3 = pd.merge(Q1,FF,on=['Month','Year'],how='outer').copy()
    Q3['excess'] = Q3['Stock_Vw_Ret'] - Q3['RF']
    
    # Make sure only include data from July 1926 to December 2023
    start = np.where((Q3['Year'] == 1926) & (Q3['Month'] == 7))[0][0]
    end = np.where((Q3['Year'] == 2023) & (Q3['Month'] == 12))[0][0]
    Q3 = Q3[start:(end+1)].copy()
    
    correlation = round(Q3['excess'].corr(Q3['MktRF']),8)
    max_diff = round(max(abs(Q3['excess']-Q3['MktRF'])),8)
    
    return [correlation,max_diff]

In [16]:
PS1_Q3(Monthly_CRSP_Stocks,FF3)

[0.99998891, 0.00269598]